<a href="https://colab.research.google.com/github/MehdiAzizian/ICT4BD/blob/main/idfedit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install eppy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.7/869.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 8.5 MB/s eta 0:00:00
  Created wheel for tinynumpy: filename=tinynumpy-1.2.1-py3-none-any.whl size=18463 sha256=cfcd69fdca32c03cfc89a850a834a971126a16a9d5c341208e315291b9df693d
  Stored in directory: /root/.cache/pip/wheels/d8/c3/c5/85bd9b13ea6e3c76b0bc39db2b28b7d743a771547bf0fb0bdd
  Created wheel for pydot3k: filename=pydot3k-1.0.17-py3-none-any.whl size=19038 sha256=6f971a9dde219b49c455ef5240acbb04086089b3913ae6b8d65545648400f2e1
  Stored in directory: /root/.cache/pip/wheels/d6/df/1f/e1f65a805151fead9e6d9fcbd1441bcf3bba677c4bb6b61059
Successfully built tinynumpy pydot3k
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.4
    Uninstalling beautifulsoup4-4.13.4:
      Successfully uninstalled beautifulsoup4-4.13.4
ERROR: pip's

In [ ]:
from eppy.modeleditor import IDF
import pandas as pd
from matplotlib import pyplot as plt

# File paths
iddfile = 'V9-6-0-Energy+.idd'
idf_file = 'Office1.idf'
weather = 'Torino_IT-hour.epw'

# Load IDF
IDF.setiddname(iddfile)
idf = IDF(idf_file, weather)


### -----------------------------
### 1. WALL & ROOF INSULATION
### -----------------------------

for mat in idf.idfobjects["MATERIAL"]:
    if mat.Name == "Wall Insulation":
        mat.Thickness = 0.25  # 25 cm
    if mat.Name == "Roof Insulation":
        mat.Thickness = 0.30  # 30 cm

### -----------------------------
### 2. WINDOW GLAZING TYPE
### -----------------------------
# Change window construction to triple glazing

for win in idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]:
    win.Construction_Name = "Triple Clear"  # Change as per your IDF

### -----------------------------
### 3. WINDOW-TO-WALL RATIO (WWR)
### -----------------------------
# Reduce WWR to 20%

from math import sqrt

walls = [s for s in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if s.Surface_Type == "Wall" and s.Outside_Boundary_Condition == "Outdoors"]
windows = idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]

for wall in walls:
    win_list = [w for w in windows if w.Building_Surface_Name == wall.Name]
    wall_area = wall.area
    for win in win_list:
        win_area = win.area
        desired_area = wall_area * 0.20 / len(win_list)
        factor = sqrt(desired_area / win_area)
        win.Length *= factor
        win.Height *= factor

### -----------------------------
### 4. LIGHTING SYSTEM EFFICIENCY
### -----------------------------
# Use efficient LED system (e.g., 5 W/m²)

for light in idf.idfobjects["LIGHTS"]:
    light.Watts_per_Zone_Floor_Area = 5.0

### -----------------------------
### 5. VENTILATIVE COOLING
### -----------------------------

for vent in idf.idfobjects["ZONEVENTILATION:WINDANDSTACKOPENAREA"]:
    vent.Schedule_Name = "Always On"
    vent.Air_Flow_Autocalculation_Method = "Flow/Zone"
    vent.Air_Flow_Rate_per_Zone_Floor_Area = 2.0  # 2 ACH
    vent.Minimum_Indoor_Temperature = 23  # Temp control
    vent.Maximum_Indoor_Temperature = 28

### -----------------------------
### 6. SHADING CONTROL
### -----------------------------

for shd in idf.idfobjects["WINDOWSHADINGCONTROL"]:
    shd.Shading_Type = "ExteriorBlind"
    shd.Control_Type = "OnIfHighSolarOnWindow"
    shd.Setpoint = 250  # GHI threshold in W/m2

### -----------------------------
### Save modified IDF
### -----------------------------
idf.saveas("idf_modified.idf")

### -----------------------------
### Run simulation
### -----------------------------
idf.run(readvars=True)

### -----------------------------
### Plot results
### -----------------------------
df = pd.read_csv("eplusout.csv")

# Adjust name below based on your zone
zone_temp_col = [col for col in df.columns if "Zone Operative Temperature" in col][0]
df[zone_temp_col].plot()
plt.title("Zone Operative Temperature")
plt.ylabel("°C")
plt.grid(True)
plt.show()

print("Max temperature:", df[zone_temp_col].max())


BadEPFieldError: unable to find field Control_Type

In [ ]:
#just shad replaced

from eppy.modeleditor import IDF
import pandas as pd
from matplotlib import pyplot as plt

# File paths
iddfile = 'V9-6-0-Energy+.idd'
idf_file = 'Office1.idf'
weather = 'Torino_IT-hour.epw'

# Load IDF
IDF.setiddname(iddfile)
idf = IDF(idf_file, weather)


### -----------------------------
### 1. WALL & ROOF INSULATION
### -----------------------------

for mat in idf.idfobjects["MATERIAL"]:
    if mat.Name == "Wall Insulation":
        mat.Thickness = 0.25  # 25 cm
    if mat.Name == "Roof Insulation":
        mat.Thickness = 0.30  # 30 cm

### -----------------------------
### 2. WINDOW GLAZING TYPE
### -----------------------------
# Change window construction to triple glazing

for win in idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]:
    win.Construction_Name = "Triple Clear"  # Change as per your IDF

### -----------------------------
### 3. WINDOW-TO-WALL RATIO (WWR)
### -----------------------------
# Reduce WWR to 20%

from math import sqrt

walls = [s for s in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if s.Surface_Type == "Wall" and s.Outside_Boundary_Condition == "Outdoors"]
windows = idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]

for wall in walls:
    win_list = [w for w in windows if w.Building_Surface_Name == wall.Name]
    wall_area = wall.area
    for win in win_list:
        win_area = win.area
        desired_area = wall_area * 0.20 / len(win_list)
        factor = sqrt(desired_area / win_area)
        win.Length *= factor
        win.Height *= factor

### -----------------------------
### 4. LIGHTING SYSTEM EFFICIENCY
### -----------------------------
# Use efficient LED system (e.g., 5 W/m²)

for light in idf.idfobjects["LIGHTS"]:
    light.Watts_per_Zone_Floor_Area = 5.0

### -----------------------------
### 5. VENTILATIVE COOLING
### -----------------------------

for vent in idf.idfobjects["ZONEVENTILATION:WINDANDSTACKOPENAREA"]:
    vent.Schedule_Name = "Always On"
    vent.Air_Flow_Autocalculation_Method = "Flow/Zone"
    vent.Air_Flow_Rate_per_Zone_Floor_Area = 2.0  # 2 ACH
    vent.Minimum_Indoor_Temperature = 23  # Temp control
    vent.Maximum_Indoor_Temperature = 28

### -----------------------------
### 6. SHADING CONTROL
### -----------------------------

for shd in idf.idfobjects["WINDOWSHADINGCONTROL"]:
    shd.Shading_Type = "ExteriorBlind"
    shd.Shading_Control_Type = "OnIfHighSolarOnWindow"
    shd.Setpoint = 250  # GHI threshold in W/m2

### -----------------------------
### Save modified IDF
### -----------------------------
idf.saveas("idf_modified.idf")

### -----------------------------
### Run simulation
### -----------------------------
idf.run(readvars=True)

### -----------------------------
### Plot results
### -----------------------------
df = pd.read_csv("eplusout.csv")

# Adjust name below based on your zone
zone_temp_col = [col for col in df.columns if "Zone Operative Temperature" in col][0]
df[zone_temp_col].plot()
plt.title("Zone Operative Temperature")
plt.ylabel("°C")
plt.grid(True)
plt.show()

print("Max temperature:", df[zone_temp_col].max())



/usr/local/EnergyPlus-9-6-0/energyplus --weather /content/Torino_IT-hour.epw --output-directory /content --idd /content/V9-6-0-Energy+.idd --readvars /content/idf_modified_168833.idf



FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/EnergyPlus-9-6-0/energyplus'